This module is a reproduce of Grimadi et al. 2020 RSE - "Flood mapping under vegetation using single SAR acquisitions" [link](https://www.sciencedirect.com/science/article/pii/S0034425719306029?via%3Dihub#bb0510)

We use GEE to process the datasets

In [18]:
import geemap
import numpy as np
import ee

In [19]:
Map= geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# Data preparation

The data we need to reproduce this result:

- SAR backscatter data HH polarized (Sentinel-1 IW mode)
- DEM (SRTM)
- Land Cover (Copernicus Global Land Cover Layers: CGLS-LC100 collection 2)
- Open water observation (JRC Monthly Water History, v1.1)

In [20]:
#study area
region_shp= '/Users/allen/OneDrive - University of Oklahoma/CRESTHH/Case study - Harvey/Houston_basin.shp'
region= geemap.shp_to_ee(region_shp)
Map.addLayer(region)

In [21]:
SAR= ee.Image('COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170818T122205_20170818T122235_006994_00C525_B566').clip(region)
Map.addLayer(SAR.select('VH'), {'min':-30, 'max':5}, 'SAR')

In [69]:
land= ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").select('discrete_classification')
land= land.map(lambda image: image.clip(region))
Map.addLayer(land,  {'min': 0.0,
  'max': 200.0,
  'palette': [
    "032f7e","02740b","02740b","8cf502","8cf502","a4da01",
    "ffbd05","ffbd05","7a5a02","f0ff0f","869b36","6091b4",
    "999999","ff4e4e","ff4e4e","ffffff","feffc0","020202",
    "020202"]},
  name='landcover' )

In [70]:
water= ee.ImageCollection('JRC/GSW1_1/MonthlyHistory').map(lambda image: image.clip(region))
Map.addLayer(water, {
  'bands': ['water'],
  'min': 0.0,
  'max': 2.0,
  'palette': ['ffffff', 'fffcb8', '0905ff']
}, 'water')

In [71]:
DEM= '/Users/allen/OneDrive - University of Oklahoma/CRESTHH/DEM_10m.tif'
